In [ ]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
from keras.callbacks import History 
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
!mkdir /csv
!cp /gdrive/My\ Drive/MachineLearning/dataframe_completo_2.csv /csv
!cp /gdrive/My\ Drive/MachineLearning/dataframe_originale.csv /csv

In [ ]:
#### variabili globali
csv_completo_2 = '/csv/dataframe_completo_2.csv' 
csv_orig = '/csv/dataframe_originale.csv' 

In [ ]:
dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
originale = pd.read_csv(csv_orig, encoding = "ISO-8859-1")

#  ritorna la parte di dataset che nel nome contiene il nome delle tuple di originale 
# (usato solo dopo aver partizionato originale)
def estendi(dataset, originale):
  res = pd.DataFrame()
  for index2 in range(len(originale)):
    temp = dataset[dataset["image_id"].str.contains(originale.iloc[index2]["image_id"])]
    res = res.append(temp)
    if index2 % 1000 == 0:
      print(index2)
  return res

# ritorna le 2 partizioni del dataframe (che non deve essere esteso) di fattori factor e 1-factor
# es. factor = 0.8
def partiziona(dataframe, factor):
  part1 = pd.DataFrame()  # factor originali
  part2 = pd.DataFrame()  # (1-factor) originali
  for elem in range(7):
    n = dataframe[dataframe['dx']==elem]['dx'].count() 
    p8 = int(n*factor)
    print("n di tipo ", elem, ":\t", n, "\tandranno in part1:\t", p8)

    blocco = dataframe[dataframe['dx']==elem]
    blocco1 = blocco.sample(n=p8)
    part1 = part1.append(blocco1)
    part2 = part2.append(blocco.drop(blocco1.index))

  print("size part1: ", len(part1))
  print("size part2: ", len(part2))

  return part1, part2

In [ ]:
# split di "originale" 80 20 (il 20 sarà il test)
orig_08, orig_02 = partiziona(originale, 0.8)

orig_08.to_csv('/csv/orig_08.csv', index=False)
orig_02.to_csv('/csv/orig_02.csv', index=False)

n di tipo  0 :	 327 	andranno in part1:	 261
n di tipo  1 :	 514 	andranno in part1:	 411
n di tipo  2 :	 1099 	andranno in part1:	 879
n di tipo  3 :	 115 	andranno in part1:	 92
n di tipo  4 :	 1113 	andranno in part1:	 890
n di tipo  5 :	 6705 	andranno in part1:	 5364
n di tipo  6 :	 142 	andranno in part1:	 113
size part1:  8010
size part2:  2005


In [ ]:
# split di "orig_08" 80 20 (80 train 20 validation)
orig_064, orig_016 = partiziona(orig_08, 0.8)

orig_064.to_csv('/csv/orig_064.csv', index=False)
orig_016.to_csv('/csv/orig_016.csv', index=False)

n di tipo  0 :	 261 	andranno in part1:	 208
n di tipo  1 :	 411 	andranno in part1:	 328
n di tipo  2 :	 879 	andranno in part1:	 703
n di tipo  3 :	 92 	andranno in part1:	 73
n di tipo  4 :	 890 	andranno in part1:	 712
n di tipo  5 :	 5364 	andranno in part1:	 4291
n di tipo  6 :	 113 	andranno in part1:	 90
size part1:  6405
size part2:  1605


In [ ]:
# dalla partizione del datset originale (orig_X) faccio la partizione del dataset esteso (esteso_X)
#esteso_02 = estendi(dataset, orig_02)
esteso_064 = estendi(dataset, orig_064)
#esteso_016 = estendi(dataset, orig_016)

#print("esteso:\t", len(dataset), "\ntrain:\t", len(esteso_064), "\nvalidation:\t", len(esteso_016), "\ntest:\t", len(esteso_02))

#esteso_02.to_csv('/csv/esteso_02.csv', index=False)
esteso_064.to_csv('/csv/esteso_064.csv', index=False)
#esteso_016.to_csv('/csv/esteso_016.csv', index=False)

0
1000
2000
3000
4000
5000
6000


In [ ]:
from google.colab import files
files.download('/csv/orig_016.csv')
files.download('/csv/orig_02.csv',)
files.download('/csv/esteso_064.csv',)